<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# [SUBSCRIBER] - Pub/Sub API - exercise solution 

In this __LAB exercise__, you will be asked to:

  - create a new publisher
  - create a new subscriber
  - send JSON messages using the publisher
  - pull the messages using the subscriber
  - delete the publisher/subscriber

### Install the libraries

In [17]:
pip install -q google-cloud

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install -q google-cloud-pubsub

E0303 16:30:08.451590066     584 backup_poller.cc:133]       Run client channel backup poller: {"created":"@1614789008.451554988","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":321,"referenced_errors":[{"created":"@1614789008.451552173","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":956,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
Note: you may need to restart the kernel to use updated packages.


### Setup the environment

In [19]:
# Import the python libraries

import os
from google.cloud import pubsub_v1

In [20]:
# Import the .json credentials and set the environment variable (TODO:student)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./credentials.json"

## Create a new Publisher (change notebook)

Before continuing, switch to the ___GCP_Pub_Sub_JSON_publisher.ipynb___ notebook and instantiate a new publisher into the `Create a new Publisher` section.

## Create a new Subscriber

__Note__: in order to avoid conflicts, please name the topic_id as "\<surname\>-python-topic-json"

__Note 2__: in order to avoid conflicts, please name the subscription_id as "\<surname\>-python-subscription-json"

In [21]:
# Set the subscriber parameters

project_id = "preparazione-lezione-gcp"
topic_id = "python-topic-json"
subscription_id = "python-subscription-json"

In [22]:
# Create the new pull subscription

publisher = pubsub_v1.PublisherClient()
subscriber = pubsub_v1.SubscriberClient()
topic_path = publisher.topic_path(project_id, topic_id)
subscription_path = subscriber.subscription_path(project_id, subscription_id)

with subscriber:
    subscription = subscriber.create_subscription(
        request={"name": subscription_path, "topic": topic_path}
    )

print(f"Subscription created: {subscription}")

Subscription created: name: "projects/preparazione-lezione-gcp/subscriptions/python-subscription-json"
topic: "projects/preparazione-lezione-gcp/topics/python-topic-json"
push_config {
}
ack_deadline_seconds: 10
message_retention_duration {
  seconds: 604800
}
expiration_policy {
  ttl {
    seconds: 2678400
  }
}



## Send JSON messages (change notebook)

Now switch to the __GCP_Pub_Sub_JSON_publisher.ipynb__ notebook and complete the code for publishing JSON messages in the `Send JSON messages` section

## Read the publisher's messages

Consume the car messages and access directly to the ___production year___ and ___color___ fields

In [23]:
import json

# Retrieve(pull) the messages from the subscriber
# Hint: use the json.loads() fuction to de-serialize

timeout = 5.0

from concurrent.futures import TimeoutError

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_id)

def callback(message):
    data = json.loads(message.data.decode('utf-8'))
    year = data ['Year']
    color = data ['Color']
    print(f"Received: Year {year} - Color {color}.")
    message.ack()

streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

with subscriber:
    try:
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        streaming_pull_future.cancel()

Listening for messages on projects/preparazione-lezione-gcp/subscriptions/python-subscription-json..

Received: Year 2019 - Color black.
Received: Year 1998 - Color white.
Received: Year 2016 - Color green.


## Clean up

In [24]:
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_id)

with subscriber:
    subscriber.delete_subscription(request={"subscription": subscription_path})

print(f"Subscription deleted: {subscription_path}.")

Subscription deleted: projects/preparazione-lezione-gcp/subscriptions/python-subscription-json.


##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2021 Quantia Consulting, srl. All rights reserved.